# Import Modules

In [4]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

# User Defined Functions

In [1]:
def artist_id_map(line):
    (id, name) = line.split('\t')
    if name in ['',' ',None]:
        return None
    else:
        try:
            return (int(id),name.strip())
        except ValueError,e:
            return None

In [2]:
def alias_map(line):
    tokens = line.split('\t')
    if tokens == []:
        return None
    if tokens[0] in ['',' ',None]:
        return None
    else:
        return (int(tokens[0]),int(tokens[1]))

In [3]:
def prepare_training_data(line):
    userID, artistID, count = map(int,line.split(' '))
    finalArtistID = bArtistAlias.value.get(artistID,artistID)
    return Rating(userID, finalArtistID, count)

# Prepare the Data

In [5]:
rawUserArtistData = sc.textFile("vagrant/profiledata_06-May-2005/user_artist_data.txt")

In [6]:
rawArtistData = sc.textFile("vagrant/profiledata_06-May-2005/artist_data.txt")

In [7]:
artistByID = rawArtistData.map(artist_id_map)

In [8]:
rawArtistAlias = sc.textFile("vagrant/profiledata_06-May-2005/artist_alias.txt")

In [9]:
artistAlias = rawArtistAlias.map(alias_map) \
                            .filter(lambda x: x is not None and None not in x) \
                            .collectAsMap()

# Building a First Model

In [10]:
bArtistAlias = sc.broadcast(artistAlias)

In [11]:
trainData = rawUserArtistData.map(prepare_training_data).cache()

In [ ]:
model = ALS.trainImplicit(ratings=trainData, rank=10, iterations=5, lambda_=0.01, alpha=1.0)